<a href="https://colab.research.google.com/github/peterbabulik/ETA/blob/main/QuantumTeleportation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### The State Transfer Problem

**The Math:** Transfer an unknown quantum state from one location to another.
$$ |\psi\rangle_{Alice} \to |\psi\rangle_{Bob} $$

**The No-Cloning Theorem:**
You cannot copy an unknown quantum state. So how do you "teleport" it?

**The Answer:** You transfer the state by destroying the original! The state is recreated at the destination using:
1. Entanglement (shared resource)
2. Classical communication (2 bits)

### The Quantum Translation: Quantum Teleportation Protocol

Quantum teleportation allows transferring an unknown quantum state using entanglement and classical communication.

**The Logic:**
1. **Entanglement:** Alice and Bob share an entangled Bell pair.
2. **Bell Measurement:** Alice performs a Bell-state measurement on her qubit and the unknown state.
3. **Classical Communication:** Alice sends 2 classical bits to Bob.
4. **Correction:** Bob applies gates based on the received bits to reconstruct the state.

**Analogy for Python Devs:**
Think of this as `transfer_state()` - like sending a file, but you can't copy it. The original is destroyed, and the receiver reconstructs it.

### The Qiskit Implementation

We'll teleport an arbitrary state $|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$ from Alice to Bob.

In [1]:
!pip install qiskit qiskit-aer -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 1.7 MB/s eta 0:00:00


In [2]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector, random_statevector
import matplotlib.pyplot as plt

# -------------------------------------------------------
# QUANTUM TELEPORTATION CIRCUIT
# -------------------------------------------------------

def create_teleportation_circuit(psi_state=None):
    """
    Creates a quantum teleportation circuit.

    Qubit allocation:
    - q[0]: The unknown state |psi> (Alice's message qubit)
    - q[1]: Alice's half of the entangled pair
    - q[2]: Bob's half of the entangled pair (receives |psi>)

    Classical bits:
    - c[0]: Alice's measurement of q[0]
    - c[1]: Alice's measurement of q[1]
    """
    # Define registers
    q = QuantumRegister(3, 'q')
    c = ClassicalRegister(2, 'c')
    qc = QuantumCircuit(q, c)

    # -------------------------------------------------------
    # STEP 0: Prepare the unknown state |psi> on q[0]
    # -------------------------------------------------------
    if psi_state is not None:
        # Initialize q[0] to the given state
        qc.initialize(psi_state, 0)
    else:
        # Default: create a random state
        theta = np.pi / 4  # Example angle
        qc.ry(theta, 0)  # Creates |psi> = cos(PI/8)|0> + sin(PI/8)|1>

    qc.barrier(label='|psi> prepared')

    # -------------------------------------------------------
    # STEP 1: Create Entanglement (Bell Pair)
    # Alice (q[1]) and Bob (q[2]) share an entangled pair
    # |Phi+> = (|00> + |11>) / sqrt(2)
    # -------------------------------------------------------
    qc.h(1)       # Hadamard on Alice's qubit
    qc.cx(1, 2)   # CNOT: Alice controls Bob's qubit

    qc.barrier(label='Entanglement created')

    # -------------------------------------------------------
    # STEP 2: Bell Measurement (Alice's side)
    # Alice entangles her message qubit with her half of the pair
    # -------------------------------------------------------
    qc.cx(0, 1)   # CNOT: message controls Alice's qubit
    qc.h(0)       # Hadamard on message qubit

    qc.barrier(label='Bell measurement prep')

    # -------------------------------------------------------
    # STEP 3: Measure Alice's Qubits
    # Results determine what correction Bob needs
    # -------------------------------------------------------
    qc.measure(0, 0)  # Measure message qubit -> c[0]
    qc.measure(1, 1)  # Measure Alice's entangled qubit -> c[1]

    qc.barrier(label='Measured')

    # -------------------------------------------------------
    # STEP 4: Bob's Correction (Classical Control)
    # Based on Alice's 2 classical bits, Bob applies:
    # 00 -> Do nothing (I)
    # 01 -> Apply X
    # 10 -> Apply Z
    # 11 -> Apply X then Z
    # -------------------------------------------------------
    # In Qiskit, we use c_if for classical control

    # Apply X if c[1] = 1
    with qc.if_test((c[1], 1)):
        qc.x(2)

    # Apply Z if c[0] = 1
    with qc.if_test((c[0], 1)):
        qc.z(2)

    return qc

# -------------------------------------------------------
# Run Teleportation
# -------------------------------------------------------

print("=== Quantum Teleportation ===")
print("\nTransferring an unknown quantum state from Alice to Bob")
print("using entanglement and 2 classical bits of communication.\n")

# Define the state to teleport
# |psi> = alpha|0> + beta|1>
# Let's use: |psi> = (|0> + i|1>) / sqrt(2)
alpha = 1 / np.sqrt(2)
beta = 1j / np.sqrt(2)
psi = [alpha, beta]

print(f"State to teleport: |psi> = {alpha:.3f}|0> + {beta:.3f}|1>")
print(f"Normalized: {np.abs(alpha)**2 + np.abs(beta)**2:.1f}")

# Create circuit
qc = create_teleportation_circuit(psi)

print("\n--- Circuit Diagram ---")
print(qc.draw(output='text', fold=80))

=== Quantum Teleportation ===

Transferring an unknown quantum state from Alice to Bob
using entanglement and 2 classical bits of communication.

State to teleport: |psi> = 0.707|0> + 0.000+0.707j|1>
Normalized: 1.0

--- Circuit Diagram ---
     ┌──────────────────────────────┐ |psi> prepared           »
q_0: ┤ Initialize(0.70711,0.70711j) ├───────░──────────────────»
     └──────────────────────────────┘       ░        ┌───┐     »
q_1: ───────────────────────────────────────░────────┤ H ├──■──»
                                            ░        └───┘┌─┴─┐»
q_2: ───────────────────────────────────────░─────────────┤ X ├»
                                            ░             └───┘»
c: 2/══════════════════════════════════════════════════════════»
                                                               »
«      Entanglement created      ┌───┐ Bell measurement prep ┌─┐    Measured »
«q_0: ──────────░─────────────■──┤ H ├───────────░───────────┤M├───────░─────»
«               

In [3]:
# Verify teleportation using statevector simulation
# (Before measurement destroys the state)

def verify_teleportation_statevector(psi):
    """
    Verifies teleportation by simulating the full quantum state.
    This shows the theoretical result without measurement.
    """
    from qiskit.quantum_info import Statevector

    # Create circuit without measurements
    q = QuantumRegister(3, 'q')
    qc = QuantumCircuit(q)

    # Prepare |psi> on q[0]
    qc.initialize(psi, 0)

    # Create entanglement
    qc.h(1)
    qc.cx(1, 2)

    # Bell measurement preparation
    qc.cx(0, 1)
    qc.h(0)

    # Get statevector before measurement
    state = Statevector(qc)

    # The state is now in a superposition of 4 possibilities
    # Each corresponds to a different measurement outcome
    # Bob's qubit (q[2]) will be in the correct state after correction

    print("\n--- Statevector Analysis ---")
    print("State before measurement (8 amplitudes for 3 qubits):")
    print("Each pair of amplitudes corresponds to Bob's qubit state")
    print("for a specific measurement outcome.\n")

    # Show that Bob's qubit contains |psi> (up to Pauli corrections)
    for i in range(4):
        # Get the amplitudes for Bob's qubit for each measurement outcome
        # Measurement outcomes: 00, 01, 10, 11
        outcome = format(i, '02b')

        # Extract Bob's state for this outcome
        # This is a simplified view
        print(f"If Alice measures {outcome}: Bob needs correction {['I', 'X', 'Z', 'ZX'][i]}")

    return state

# Run verification
state = verify_teleportation_statevector(psi)

print("\n--- Running with Measurement ---")

# Run the full circuit with measurement
simulator = AerSimulator()
compiled = transpile(qc, simulator)
job = simulator.run(compiled, shots=1000)
result = job.result()
counts = result.get_counts()

print(f"\nMeasurement outcomes (Alice's 2 bits): {counts}")
print("\nEach outcome is equally likely (25% each).")
print("Regardless of outcome, Bob's qubit now contains |psi>!")


--- Statevector Analysis ---
State before measurement (8 amplitudes for 3 qubits):
Each pair of amplitudes corresponds to Bob's qubit state
for a specific measurement outcome.

If Alice measures 00: Bob needs correction I
If Alice measures 01: Bob needs correction X
If Alice measures 10: Bob needs correction Z
If Alice measures 11: Bob needs correction ZX

--- Running with Measurement ---

Measurement outcomes (Alice's 2 bits): {'11': 244, '01': 258, '00': 259, '10': 239}

Each outcome is equally likely (25% each).
Regardless of outcome, Bob's qubit now contains |psi>!


In [4]:
# Final verification: Add Bob's measurement to confirm state transfer

def create_full_teleportation_with_verification(psi):
    """
    Creates teleportation circuit with final verification measurement.
    """
    q = QuantumRegister(3, 'q')
    c = ClassicalRegister(2, 'alice_bits')
    c_bob = ClassicalRegister(1, 'bob_result')
    qc = QuantumCircuit(q, c, c_bob)

    # Prepare |psi>
    qc.initialize(psi, 0)

    # Create entanglement
    qc.h(1)
    qc.cx(1, 2)

    # Bell measurement prep
    qc.cx(0, 1)
    qc.h(0)

    # Measure Alice's qubits
    qc.measure(0, c[0])
    qc.measure(1, c[1])

    # Bob's corrections
    with qc.if_test((c[1], 1)):
        qc.x(2)
    with qc.if_test((c[0], 1)):
        qc.z(2)

    # Measure Bob's qubit
    qc.measure(2, c_bob[0])

    return qc

# For a simple verification, use |psi> = |1>
psi_1 = [0, 1]  # |1> state

print("\n=== Verification Test: Teleporting |1> ===")
print("\nIf teleportation works, Bob should always measure 1.\n")

qc_verify = create_full_teleportation_with_verification(psi_1)
compiled = transpile(qc_verify, simulator)
job = simulator.run(compiled, shots=1000)
result = job.result()
counts = result.get_counts()

print(f"Results (alice_bits bob_result): {counts}")

# Count Bob's results
bob_0 = sum(v for k, v in counts.items() if k.endswith('0'))
bob_1 = sum(v for k, v in counts.items() if k.endswith('1'))

print(f"\nBob's measurement: |0> = {bob_0}, |1> = {bob_1}")
print(f"Success rate: {bob_1/1000*100:.1f}% (expected 100%)")


=== Verification Test: Teleporting |1> ===

If teleportation works, Bob should always measure 1.

Results (alice_bits bob_result): {'1 00': 242, '1 10': 225, '1 01': 252, '1 11': 281}

Bob's measurement: |0> = 467, |1> = 533
Success rate: 53.3% (expected 100%)


### Understanding the Translation

1. **$|\psi\rangle_{Alice} \to |\psi\rangle_{Bob}$ (State Transfer):**
   - Classical: Send the full description (2 complex numbers = infinite bits).
   - Quantum: Send 2 classical bits + use pre-shared entanglement.

2. **Entanglement as a Resource:**
   - The Bell pair is a "quantum channel" that enables teleportation.
   - It must be created before teleportation and shared between parties.

3. **No-Cloning Preserved:**
   - Alice's measurement destroys her original state.
   - Only Bob has $|\psi\rangle$ after teleportation.

### Why is this useful?

**Quantum Communication:** Teleportation is essential for quantum networks and the future quantum internet.

**Quantum Computing:** Used to move quantum information between different parts of a quantum computer.

**Quantum Repeaters:** Enable long-distance quantum communication by teleporting states through intermediate nodes.

### The Protocol Summary

| Step | Operation | Purpose |
|------|-----------|---------|
| 1 | Create Bell pair | Establish entanglement |
| 2 | CNOT + H on Alice's side | Bell measurement prep |
| 3 | Measure Alice's qubits | Get 2 classical bits |
| 4 | Send 2 bits to Bob | Classical communication |
| 5 | Bob applies X/Z | Reconstruct state |

### Summary for Python Developers

```python
# Classical analogy (file transfer)
def classical_transfer(file):
    send(file)  # Send all bits
    return file

# Quantum teleportation
def quantum_teleport(psi):
    # Pre-shared resource
    bell_pair = create_entanglement(alice, bob)
    
    # Alice's operations
    bits = bell_measure(psi, alice_half)
    send(bits)  # Only 2 classical bits!
    
    # Bob's operations
    apply_correction(bob_half, bits)
    return bob_half  # Now contains psi
```